In [1]:
from QCloud import *
import numpy as np
import matplotlib.pyplot as plt

def sum_nested_list(nested_list):
    total_sum = 0
    for item in nested_list:
        if isinstance(item, list):  # If the item is a list, sum its elements
            total_sum += sum(item)
        else:  # If it's a number, add it directly
            total_sum += item
    return total_sum

def calculate_fidelity(job_records):
    num_of_devices = []
    fidelity = []
    for job_id, events in job_records.items():
        # print(f"Job ID: {job_id}, Events: {events}")
        num_of_devices.append(len(events['devc_proc']))
        fidelity.append(events['fidelity'])
    print(f'fidelity: {round(np.mean(fidelity), 5)} +/- {round(np.std(fidelity), 5)}')
    return num_of_devices, fidelity
    
def calculate_comm_time(job_records): 
    comm_time = []
    for job_id, events in job_records.items():
        # print(f"Job ID: {job_id}, Events: {events}")
        comm_time.append(events['comm_time'])
    print(f'Comm time: {round(sum_nested_list(comm_time), 5)}')
    
# deploying jobs from CSV file. 
allocation_mode = ['rlbase']
num_of_devices = {}
fidelity_list = {}
PRINTLOG = False

for al_mode in allocation_mode: 
    # Create quantum devices
    ibm_strasbourg = IBM_Strasbourg(env=None, name="ibm_strasbourg", cali_filepath='QCloud/calibration/03_2025/ibm_strasbourg_calibrations_2025-03-29T05_16_26Z.csv', printlog = PRINTLOG)
    ibm_brussels = IBM_Brussels(env=None, name="ibm_brussels", cali_filepath='QCloud/calibration/03_2025/ibm_brussels_calibrations_2025-03-30T02_20_13Z.csv', printlog = PRINTLOG)
    ibm_kyiv = IBM_Kyiv(env=None, name="ibm_kyiv", cali_filepath='QCloud/calibration/03_2025/ibm_kyiv_calibrations_2025-03-30T02_24_45Z.csv', printlog = PRINTLOG)
    ibm_quebec = IBM_Quebec(env=None, name="ibm_quebec", cali_filepath='QCloud/calibration/03_2025/ibm_quebec_calibrations_2025-03-30T02_51_33Z.csv', printlog = PRINTLOG)
    ibm_kawasaki = IBM_Kawasaki(env=None, name="ibm_kawasaki", cali_filepath='QCloud/calibration/03_2025/ibm_kawasaki_calibrations_2025-03-30T03_15_05Z.csv', printlog = PRINTLOG)

    devices = [ibm_strasbourg, ibm_brussels, ibm_kyiv, ibm_quebec, ibm_kawasaki]

    # Initialize and run the simulation using jobs from a CSV file
    qcloudsimenv = QCloudSimEnv(
        devices=devices,
        broker_class=ParallelBroker,
        job_feed_method="dispatcher",
        file_path="synth_job_batches/1-large-circuits.csv", 
        allocation_mode=al_mode,
        printlog = PRINTLOG
    )
    qcloudsimenv.run()
    print(f'Allocation mode: {al_mode}')
    print(f'Total Sim-time: {qcloudsimenv.now}')

    # Calculate Fidelity and communication time
    # Access job records after the simulation
    job_records = qcloudsimenv.job_records_manager.get_job_records()
    num_of_devices[al_mode], fidelity_list[al_mode] = calculate_fidelity(job_records)
    calculate_comm_time(job_records)
    print('---------------------------')

Allocation mode: rlbase
Total Sim-time: 328.7690804597701
fidelity: 0.6123 +/- 0.0
Comm time: 6.92
---------------------------
